In [1]:
%load_ext autoreload
%autoreload

!rm -r x86_64
!nrnivmodl mechanisms

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import pprint
pp = pprint.PrettyPrinter(indent=2)

import matplotlib.pyplot as plt
%matplotlib notebook
import MEAutility as mu

import json
import numpy
import time
import numpy as np

import LFPy

Creating x86_64 directory for .o files.

/gpfs/bbp.cscs.ch/home/damart/LFPy/BluePyOpt/examples/l5pc_LFPy
mechanisms/CaDynamics_E2.mod mechanisms/Ca_HVA.mod mechanisms/Ca_LVAst.mod mechanisms/Ih.mod mechanisms/Im.mod mechanisms/K_Pst.mod mechanisms/K_Tst.mod mechanisms/Nap_Et2.mod mechanisms/NaTa_t.mod mechanisms/NaTs2_t.mod mechanisms/ProbAMPANMDA_EMS.mod mechanisms/ProbGABAAB_EMS.mod mechanisms/SK_E2.mod mechanisms/SKv3_1.mod
CaDynamics_E2.mod Ca_HVA.mod Ca_LVAst.mod Ih.mod Im.mod K_Pst.mod K_Tst.mod Nap_Et2.mod NaTa_t.mod NaTs2_t.mod ProbAMPANMDA_EMS.mod ProbGABAAB_EMS.mod SK_E2.mod SKv3_1.mod
"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/x86_64/bin/nocmodl" CaDynamics_E2
"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/x86_64/bin/nocmodl" Ca_HVA
"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64

"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/x86_64/bin/nocmodl" NaTa_t
Translating NaTa_t.mod into NaTa_t.c
Thread Safe
"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/share/nrn/libtool" --tag=CC --mode=compile /gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/tools/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/hpe-mpi-2.21-7pbszh6v5u/bin/mpicc -DHAVE_CONFIG_H  -I. -I.. -I"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/include/nrn" -I"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/x86_64/lib"      -O2 -g -c -o NaTa_t.lo NaTa_t.c
"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/x86_64/bin/nocmodl" NaTs2_t
"/gpfs/bbp

"/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/libraries/2020-02-01/linux-rhel7-x86_64/intel-19.0.4/neuron-7.8.0b-3lvust5k7q/share/nrn/libtool" --tag=CC --mode=link /gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/tools/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/hpe-mpi-2.21-7pbszh6v5u/bin/mpicc -module  -O2 -g    -L/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/external-libraries/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/readline-8.0-uuhvgs2hgl/lib -Wl,-rpath,/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/external-libraries/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/readline-8.0-uuhvgs2hgl/lib -L/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/external-libraries/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/ncurses-6.1-esk7yfugjx/lib -Wl,-rpath,/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/external-libraries/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/ncurses-6.1-esk7yfugjx/lib -Wl,-rpath,/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/tools/2020-02-01/linux-rhel7-x86_64/gcc-8.3.0/hpe-mpi-2.21-7pbszh6v5u/lib -o 

## Define extracellular electrodes

In [2]:
mea_positions = np.zeros((5, 3))
mea_positions[:, 2] = 20
mea_positions[:, 1] = np.linspace(0, 900, 5)
probe = mu.return_mea(info={'pos': mea_positions, 'center': False, 'plane': 'xy'})
electrode = LFPy.RecExtElectrode(probe=probe, method='linesource')

## Setup protocols, fitness, and evaluator

In [3]:
import l5pc_model
import l5pc_evaluator

feature_set = "extra" # 'soma'/'bap'

morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = l5pc_model.define_parameters()
mechanisms = l5pc_model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      

if feature_set == "extra":
    fitness_protocols = l5pc_evaluator.define_protocols(electrode) 
else:
    fitness_protocols = l5pc_evaluator.define_protocols() 

fitness_calculator = l5pc_evaluator.define_fitness_calculator(fitness_protocols, feature_set=feature_set, 
                                                              probe=probe)

if feature_set == "extra":
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)
else:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)

evaluator = ephys.evaluators.CellEvaluator(                                          
                cell_model=l5pc_cell,                                                       
                param_names=param_names,                                                    
                fitness_protocols=fitness_protocols,                                        
                fitness_calculator=fitness_calculator,                                      
                sim=sim) 

In [4]:
extrafeat = fitness_calculator.objectives[-1].features[0]

In [5]:
release_params = {
    'gNaTs2_tbar_NaTs2_t.apical': 0.026145,
    'gSKv3_1bar_SKv3_1.apical': 0.004226,
    'gImbar_Im.apical': 0.000143,
    'gNaTa_tbar_NaTa_t.axonal': 3.137968,
    'gK_Tstbar_K_Tst.axonal': 0.089259,
    'gamma_CaDynamics_E2.axonal': 0.002910,
    'gNap_Et2bar_Nap_Et2.axonal': 0.006827,
    'gSK_E2bar_SK_E2.axonal': 0.007104,
    'gCa_HVAbar_Ca_HVA.axonal': 0.000990,
    'gK_Pstbar_K_Pst.axonal': 0.973538,
    'gSKv3_1bar_SKv3_1.axonal': 1.021945,
    'decay_CaDynamics_E2.axonal': 287.198731,
    'gCa_LVAstbar_Ca_LVAst.axonal': 0.008752,
    'gamma_CaDynamics_E2.somatic': 0.000609,
    'gSKv3_1bar_SKv3_1.somatic': 0.303472,
    'gSK_E2bar_SK_E2.somatic': 0.008407,
    'gCa_HVAbar_Ca_HVA.somatic': 0.000994,
    'gNaTs2_tbar_NaTs2_t.somatic': 0.983955,
    'decay_CaDynamics_E2.somatic': 210.485284,
    'gCa_LVAstbar_Ca_LVAst.somatic': 0.000333
}

In [6]:
t_start = time.time()
LFPy_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), 
                                         param_values=release_params)
t_end = time.time()
print(t_end-t_start)

Exception: Traceback (most recent call last):
  File "/gpfs/bbp.cscs.ch/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/protocols.py", line 191, in _run_func
    for recording in self.recordings}
  File "/gpfs/bbp.cscs.ch/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/protocols.py", line 191, in <dictcomp>
    for recording in self.recordings}
  File "/gpfs/bbp.cscs.ch/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/recordings.py", line 149, in response
    self.electrode.LFP)
AttributeError: 'NoneType' object has no attribute 'LFP'


In [ ]:
feat, mean_wf = extrafeat.calculate_feature(LFPy_responses, return_waveforms=True)

In [ ]:
tr, pk = bpopt.ephys.extra_features_utils._get_trough_and_peak_idx(mean_wf, False)

In [ ]:
plt.figure()
cmap = plt.get_cmap('viridis')
idxs = range(0, 10)
for i, mw in enumerate(mean_wf):
    if i in idxs:
        color = cmap(i/len(mean_wf))
        plt.plot(mw / np.max(np.abs(mw)), color=color, lw=0.3)
        plt.axvline(tr[i], ls='--', color=color)
        plt.axvline(pk[i], ls='-.', color=color)

In [ ]:
mean_wf_norm = mean_wf / np.max(np.abs(mean_wf), 1, keepdims=True)

In [ ]:
mu.plot_mea_recording(mean_wf_norm, probe)

## Optimization

In [ ]:
offspring_size = 20
max_ngen = 20

In [ ]:
opt = bpopt.optimisations.DEAPOptimisation(                                     
    evaluator=evaluator,                                                            
    offspring_size=offspring_size) 
final_pop, halloffame, log, hist = opt.run(max_ngen=max_ngen, cp_filename='checkpoints/checkpoint.pkl')

In [ ]:
# import pickle
# original_responses = pickle.load(open("responses.pkl", "rb"))

# fig, axes = plt.subplots(len(original_responses), figsize=(10,10))
# for index, (resp_name, response) in enumerate(sorted(original_responses.items())):
#     axes[index].plot(response['time'], response['voltage'], label=resp_name, lw=1.0, alpha=0.5)
#     axes[index].plot(LFPy_responses[resp_name]['time'], LFPy_responses[resp_name]['voltage'], label=resp_name, lw=1.0, alpha=0.5)
#     axes[index].set_title(resp_name)
# fig.tight_layout()

# for resp_name in original_responses:
#     end = numpy.min([len(original_responses[resp_name]['time']), len(LFPy_responses[resp_name]['time'])])
#     _ = numpy.sum(numpy.abs(original_responses[resp_name]['time'][:end]-LFPy_responses[resp_name]['time'][:end]))
#     print(resp_name, _/end)
#     print(LFPy_responses[resp_name]['time'].iloc[-1], original_responses[resp_name]['time'].iloc[-1])
# fig.show()

In [ ]:
# fig, axes = plt.subplots(len(original_responses), figsize=(10,10))
# for index, (resp_name, response) in enumerate(sorted(original_responses.items())): 
        
#         diff = numpy.diff(original_responses[resp_name]['time'])
#         axes[index].plot(original_responses[resp_name]['time'][:-1], diff, lw=1.0, alpha=0.7)
        
#         diff = numpy.diff(LFPy_responses[resp_name]['time'])
#         axes[index].plot(LFPy_responses[resp_name]['time'][:-1], diff, lw=1.0, alpha=0.7)
        
#         axes[index].set_xlabel("Time (ms)")
#         axes[index].set_ylabel("dt (ms)")